# Load Libraries

In [3]:
import boto3
import sagemaker
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import sagemaker
from sagemaker import get_execution_role
from sagemaker.automl.automl import AutoML
import tarfile
import os
import time
from datetime import datetime
from time import strftime
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sklearn.neural_network import MLPClassifier
from sagemaker.sklearn import SKLearn
import joblib

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sklearn.metrics import confusion_matrix

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [4]:
from sklearn.linear_model import LogisticRegression

In [22]:
from sklearn.preprocessing import LabelEncoder

In [18]:
#!pip install xgboost
import xgboost as xgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 2.4 MB/s eta 0:00:00:00:0100:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Open S3 Session

In [5]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

bucket_name = bucket

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Read in Training

In [6]:
# Specify the S3 bucket name and file path
bucket_name = bucket
file_path = 'ADS508_project/cleandata/X_train_prep.csv'

# Read CSV file from S3 bucket into DataFrame
X_train_prep = pd.read_csv(f's3://{bucket_name}/{file_path}')

X_train_prep.head()

,0,1,2,3,4,5,6,7,8,9,...,state_id_TX,state_id_UK,state_id_UT,state_id_VA,state_id_VT,state_id_WA,state_id_WI,state_id_WV,state_id_WY,outcome
0,-0.085612,-0.192749,0.032051,0.016743,-0.010959,-0.208290,-0.152666,0.297640,-0.075811,-0.289037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biden_nega
1,-0.212201,-0.275853,0.037405,0.060544,-0.318471,-0.185872,-0.159038,0.012011,0.002532,-0.199857,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biden_nega
2,0.086982,0.119298,0.000721,0.070849,0.078040,-0.039077,-0.055840,0.138417,-0.041773,-0.203287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biden_nega
3,-0.049801,-0.023380,0.220462,0.187448,0.186836,-0.236208,-0.012966,0.230729,0.106897,-0.068943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biden_nega
4,0.043111,-0.095598,0.186926,0.210929,0.124437,-0.180557,0.122059,0.101528,-0.059285,-0.112962,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biden_nega


# Read in Validation

In [8]:
# Specify the S3 bucket name and file path
bucket_name = bucket
file_path = 'ADS508_project/cleandata/X_val_prep.csv'

# Read CSV file from S3 bucket into DataFrame
X_val_prep = pd.read_csv(f's3://{bucket_name}/{file_path}')

X_val_prep.head()

,0,1,2,3,4,5,6,7,8,9,...,state_id_TX,state_id_UK,state_id_UT,state_id_VA,state_id_VT,state_id_WA,state_id_WI,state_id_WV,state_id_WY,outcome
0,-0.020926,0.038315,-0.015478,0.129784,-0.072584,-0.225882,0.116951,0.300709,-0.293697,-0.131742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Biden_posi
1,-0.119196,-0.174804,-0.049492,0.200089,0.020939,-0.033172,-0.113429,0.294107,-0.243626,0.038364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trump_posi
2,0.023994,0.048203,0.132835,0.116420,0.221791,-0.144960,-0.026205,0.083955,0.105637,-0.224725,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trump_neut
3,0.075427,0.040858,0.099023,-0.004378,0.034570,-0.170367,0.052725,0.375484,0.045429,-0.236863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trump_posi
4,0.127470,-0.034008,0.087723,0.135007,0.221004,-0.149747,0.064742,0.260640,0.016842,0.024366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Trump_posi


# Read in Testing

In [11]:
# Specify the S3 bucket name and file path
bucket_name = bucket
file_path = 'ADS508_project/cleandata/X_test_prep.csv'

# Read CSV file from S3 bucket into DataFrame
X_test_prep = pd.read_csv(f's3://{bucket_name}/{file_path}')

X_test_prep.head()

,0,1,2,3,4,5,6,7,8,9,...,state_id_TX,state_id_UK,state_id_UT,state_id_VA,state_id_VT,state_id_WA,state_id_WI,state_id_WV,state_id_WY,outcome
0,-0.148748,-0.152604,0.410740,0.202116,0.018050,-0.217776,-0.022319,0.141656,0.011170,-0.119315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trump_neut
1,-0.028749,-0.105872,0.180524,0.218901,0.207626,-0.304348,-0.144124,0.339819,0.003144,-0.105429,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Trump_posi
2,-0.045923,-0.106403,-0.000612,0.198213,0.093984,-0.156729,-0.039995,0.291425,-0.124625,-0.062035,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trump_posi
3,0.203205,-0.088243,0.242572,-0.091901,0.203277,-0.030754,0.085939,0.190463,-0.022583,-0.079250,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Trump_neut
4,-0.053510,0.018920,0.234078,0.105144,0.229481,-0.106339,-0.051445,0.502994,-0.241178,-0.332092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trump_posi


# Baseline Model: Logistic Regression with All Data

In [7]:
# Initialize the Logistic Regression model
log_reg2 = LogisticRegression(max_iter=1000, solver='lbfgs')

# Fit the model on the balanced text training data, excluding outcome variable and 1 dummy variable
# Outcome variable is last column of training data
log_reg2.fit(X_train_prep.iloc[:, :-2], X_train_prep.iloc[:, -1])

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [9]:
# Predict on the validation set
y_val_pred = log_reg2.predict(X_val_prep.iloc[:, :-2])

In [10]:
# Evaluate the model
print("Baseline: Validation Set Accuracy:", accuracy_score(X_val_prep.iloc[:, -1], y_val_pred))
print("\nBaselin: Validation Set Classification Report:\n", classification_report(X_val_prep.iloc[:, -1], y_val_pred))

Baseline: Validation Set Accuracy: 0.6034037462958431

Baselin: Validation Set Classification Report:
               precision    recall  f1-score   support

  Biden_nega       0.35      0.64      0.46      1128
  Biden_neut       0.73      0.68      0.71      2427
  Biden_posi       0.70      0.62      0.66      2975
  Trump_nega       0.46      0.55      0.50      7729
  Trump_neut       0.65      0.68      0.66      9696
  Trump_posi       0.68      0.55      0.61     12828

    accuracy                           0.60     36783
   macro avg       0.59      0.62      0.60     36783
weighted avg       0.62      0.60      0.61     36783



In [12]:
# Predict on the test set
y_test_pred = log_reg2.predict(X_test_prep.iloc[:, :-2])
print("Baseline: Test Set Accuracy:", accuracy_score(X_test_prep.iloc[:, -1], y_test_pred))
print("\nBaselin: Test Set Classification Report:\n", classification_report(X_test_prep.iloc[:, -1], y_test_pred))

Baseline: Test Set Accuracy: 0.6075195737277077

Baselin: Test Set Classification Report:
               precision    recall  f1-score   support

  Biden_nega       0.33      0.61      0.43      1131
  Biden_neut       0.73      0.69      0.71      2569
  Biden_posi       0.68      0.61      0.65      2971
  Trump_nega       0.47      0.55      0.51      7728
  Trump_neut       0.65      0.69      0.67      9705
  Trump_posi       0.69      0.56      0.62     12680

    accuracy                           0.61     36784
   macro avg       0.59      0.62      0.60     36784
weighted avg       0.62      0.61      0.61     36784



# Baseline Logistic Regression without location data (columns 1 to 768)

In [13]:
# Initialize the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, solver='lbfgs')

# Fit the model on the balanced text training data only, BERT vectors are columns 1-768
# Outcome variable is last column of training data
log_reg.fit(X_train_prep.iloc[:, :768], X_train_prep.iloc[:, -1])

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [14]:
# Predict on the validation set
y_val_pred = log_reg.predict(X_val_prep.iloc[:, :768])

In [15]:
# Evaluate the model
print("Baseline: Validation Set Accuracy:", accuracy_score(X_val_prep.iloc[:, -1], y_val_pred))
print("\nBaselin: Validation Set Classification Report:\n", classification_report(X_val_prep.iloc[:, -1], y_val_pred))

Baseline: Validation Set Accuracy: 0.6050893075605579

Baselin: Validation Set Classification Report:
               precision    recall  f1-score   support

  Biden_nega       0.36      0.64      0.46      1128
  Biden_neut       0.73      0.68      0.71      2427
  Biden_posi       0.70      0.62      0.66      2975
  Trump_nega       0.47      0.55      0.51      7729
  Trump_neut       0.65      0.68      0.67      9696
  Trump_posi       0.68      0.56      0.61     12828

    accuracy                           0.61     36783
   macro avg       0.60      0.62      0.60     36783
weighted avg       0.62      0.61      0.61     36783



In [16]:
# Predict on the test set
y_test_pred = log_reg.predict(X_test_prep.iloc[:, :768])
print("Baseline: Test Set Accuracy:", accuracy_score(X_test_prep.iloc[:, -1], y_test_pred))
print("\nBaselin: Test Set Classification Report:\n", classification_report(X_test_prep.iloc[:, -1], y_test_pred))

Baseline: Test Set Accuracy: 0.6073020878642889

Baselin: Test Set Classification Report:
               precision    recall  f1-score   support

  Biden_nega       0.33      0.61      0.43      1131
  Biden_neut       0.73      0.69      0.71      2569
  Biden_posi       0.69      0.61      0.65      2971
  Trump_nega       0.47      0.55      0.51      7728
  Trump_neut       0.65      0.70      0.67      9705
  Trump_posi       0.69      0.56      0.62     12680

    accuracy                           0.61     36784
   macro avg       0.59      0.62      0.60     36784
weighted avg       0.62      0.61      0.61     36784



# SageMaker AutoPilot

In [ ]:
role = get_execution_role()
session = sagemaker.Session()

# Replace 'your_dataset.csv' with the name of your CSV file
input_data = 's3://508group/ADS508_project/cleandata/df_final.csv'

autopilot_job = AutoML(
    role=role,
    target_attribute_name='sentimentoutcome',  # This is the column you're predicting
    output_path='s3://508group/ADS508_project/output/autopilot/',
    max_candidates=20,
    sagemaker_session=session,
    problem_type='MulticlassClassification',  # Assuming sentimentoutcome is multiclass
    job_objective={'MetricName': 'Accuracy'}  # You can choose another metric if it fits better
)

autopilot_job.fit(inputs=input_data, wait=False, logs=True)

In [ ]:
# Initialize the S3 client
s3 = boto3.client('s3')

# S3 bucket and object key
bucket_name = '508group'
object_key = 'ADS508_project/output/autopilot/automl-2024-03-30-19-51-00-981/data-processor-models/automl-2024-03-30-19-51-00-981-dpp9-1-89acfb1b856c4754873c58ddd/output/model.tar.gz'

# Local directory to extract the contents
extract_dir = "ADS508_project/output/autopilot/automl-2024-03-30-19-51-00-981/data-processor-models/automl-2024-03-30-19-51-00-981-dpp9-1-89acfb1b856c4754873c58ddd/output/model.tar.gz"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Download the file from S3
local_file_path = os.path.join(extract_dir, 'model.tar.gz')
s3.download_file(bucket_name, object_key, local_file_path)

# Open the tar file and extract the contents
with tarfile.open(local_file_path, "r:gz") as tar:
    tar.extractall(path=extract_dir)

# After extraction, you can further process or use the contents of the extracted files
# For example, list the extracted files
extracted_files = os.listdir(extract_dir)
print("Extracted files:", extracted_files)

In [ ]:
# Download the file from S3
local_file_path = os.path.join(extract_dir, 'model.tar.gz')
s3.download_file(bucket_name, object_key, local_file_path)

# Extract the contents of the tar.gz file
with tarfile.open(local_file_path, "r:gz") as tar:
    tar.extractall(path=extract_dir)


In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")
role = get_execution_role()
session = sagemaker.Session()
# Replace 'your_dataset.csv' with the name of your CSV file
#input_data = 's3://508group/ADS508_project/cleandata/df_final.csv'
input_data = 's3://sagemaker-us-east-1-851725336500/ADS508_project/cleandata/final_data.csv'
autopilot_job = AutoML(
    role=role,
    target_attribute_name='sentimentoutcome',  # This is the column you're predicting
    #output_path='s3://508group/ADS508_project/output/autopilot/',
    output_path= 's3://sagemaker-us-east-1-851725336500/ADS508_project/output/autopilot/',
    max_candidates=10,
    sagemaker_session=session,
    problem_type='MulticlassClassification',  # Assuming sentimentoutcome is multiclass
    job_objective={'MetricName': 'Accuracy'}  # You can choose another metric if it fits better
)
autopilot_job.fit(inputs=input_data, wait=True, logs=True)
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")


# Ensemble Models

In [19]:
# Define S3 prefix
prefix = 'ADS508_project/cleandata/'

In [24]:
# Load the training data
train_data = X_train_prep

# Extract features and target variable
X_train = train_data.iloc[:, :-1]  # Features
y_train = train_data.iloc[:, -1]   # Target variable

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)

# Define hyperparameters
hyperparameters = {
    "max_depth": 6,
    "lambda": 0.015482990568956176,
    "min_child_weight": 0.0016695027194522027,
    "objective": "multi:softprob",
    "num_class": len(label_encoder.classes_)  # Number of classes for multi-class classification
}

# Train the XGBoost model
xgb_model = xgb.train(params=hyperparameters, dtrain=dtrain)

/opt/conda/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [00:41:14] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)


In [25]:
# Save the model
xgb_model.save_model('xgboost_model.model')

/opt/conda/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [00:42:05] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


# Get Labels Outputs

In [26]:
# Get the mapping of original class labels to encoded numerical labels
class_labels = label_encoder.classes_

# Print the mapping
for encoded_label, original_label in enumerate(class_labels):
    print(f"Encoded label {encoded_label}: Original label {original_label}")

Encoded label 0: Original label Biden_nega
Encoded label 1: Original label Biden_neut
Encoded label 2: Original label Biden_posi
Encoded label 3: Original label Trump_nega
Encoded label 4: Original label Trump_neut
Encoded label 5: Original label Trump_posi


## Get Performance Predictions

In [29]:
# Load the validation and testing data
val_data = X_val_prep
test_data = X_test_prep

# Extract features and target variable for validation and testing sets
X_val = val_data.iloc[:, :-1]  # Features for validation set
y_val = val_data.iloc[:, -1]   # Target variable for validation set
X_test = test_data.iloc[:, :-1]  # Features for testing set
y_test = test_data.iloc[:, -1]   # Target variable for testing set


y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

# Convert validation and testing data into DMatrix format
dval = xgb.DMatrix(X_val, label=y_val_encoded)
dtest = xgb.DMatrix(X_test, label=y_test_encoded)

# Load the trained model
xgb_model = xgb.Booster()
xgb_model.load_model('xgboost_model.model')

# Make predictions on validation and testing data
y_val_pred = xgb_model.predict(dval)
y_test_pred = xgb_model.predict(dtest)

# Evaluate the model's performance on validation set
val_accuracy = accuracy_score(y_val_encoded, y_val_pred.argmax(axis=1))
val_classification_report = classification_report(y_val_encoded, y_val_pred.argmax(axis=1))

# Evaluate the model's performance on testing set
test_accuracy = accuracy_score(y_test_encoded, y_test_pred.argmax(axis=1))
test_classification_report = classification_report(y_test_encoded, y_test_pred.argmax(axis=1))

# Print performance metrics
print("Validation Set Accuracy:", val_accuracy)
print("Validation Set Classification Report:\n", val_classification_report)
print("Testing Set Accuracy:", test_accuracy)
print("Testing Set Classification Report:\n", test_classification_report)


Validation Set Accuracy: 0.48642035723024224
Validation Set Classification Report:
               precision    recall  f1-score   support

           0       0.19      0.47      0.27      1128
           1       0.51      0.65      0.57      2427
           2       0.47      0.50      0.48      2975
           3       0.39      0.45      0.42      7729
           4       0.56      0.61      0.59      9696
           5       0.60      0.38      0.46     12828

    accuracy                           0.49     36783
   macro avg       0.45      0.51      0.47     36783
weighted avg       0.52      0.49      0.49     36783

Testing Set Accuracy: 0.48363418877772946
Testing Set Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.43      0.25      1131
           1       0.53      0.66      0.59      2569
           2       0.45      0.48      0.46      2971
           3       0.38      0.44      0.41      7728
           4       0.56 

In [30]:
# Load the training data
train_data = X_train_prep

# Extract features and target variable
X_train = train_data.iloc[:, :-1]  # Features
y_train = train_data.iloc[:, -1]   # Target variable

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)

# Define hyperparameters
hyperparameters = {
    "max_depth": 6,
    "learning_rate": 0.3,
    "n_estimators": 100,
    "objective": "multi:softprob",
    "num_class": len(label_encoder.classes_),  # Number of classes for multi-class classification
    "gamma": 0,
    "min_child_weight": 1,
    "subsample": 1,
    "colsample_bytree": 1,
    "reg_alpha": 0,
    "reg_lambda": 1
}

# Train the XGBoost model
xgb_model = xgb.train(params=hyperparameters, dtrain=dtrain)

/opt/conda/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [00:46:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [31]:
# Save the model
xgb_model.save_model('xgboost_model.model')

/opt/conda/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [00:47:25] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [32]:
# Load the validation and testing data
val_data = X_val_prep
test_data = X_test_prep

# Extract features and target variable for validation and testing sets
X_val = val_data.iloc[:, :-1]  # Features for validation set
y_val = val_data.iloc[:, -1]   # Target variable for validation set
X_test = test_data.iloc[:, :-1]  # Features for testing set
y_test = test_data.iloc[:, -1]   # Target variable for testing set


y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

# Convert validation and testing data into DMatrix format
dval = xgb.DMatrix(X_val, label=y_val_encoded)
dtest = xgb.DMatrix(X_test, label=y_test_encoded)

# Load the trained model
xgb_model = xgb.Booster()
xgb_model.load_model('xgboost_model.model')

# Make predictions on validation and testing data
y_val_pred = xgb_model.predict(dval)
y_test_pred = xgb_model.predict(dtest)

# Evaluate the model's performance on validation set
val_accuracy = accuracy_score(y_val_encoded, y_val_pred.argmax(axis=1))
val_classification_report = classification_report(y_val_encoded, y_val_pred.argmax(axis=1))

# Evaluate the model's performance on testing set
test_accuracy = accuracy_score(y_test_encoded, y_test_pred.argmax(axis=1))
test_classification_report = classification_report(y_test_encoded, y_test_pred.argmax(axis=1))

# Print performance metrics
print("Validation Set Accuracy:", val_accuracy)
print("Validation Set Classification Report:\n", val_classification_report)
print("Testing Set Accuracy:", test_accuracy)
print("Testing Set Classification Report:\n", test_classification_report)

Validation Set Accuracy: 0.48767093494277247
Validation Set Classification Report:
               precision    recall  f1-score   support

           0       0.20      0.47      0.28      1128
           1       0.51      0.67      0.58      2427
           2       0.47      0.50      0.48      2975
           3       0.39      0.45      0.41      7729
           4       0.56      0.61      0.59      9696
           5       0.60      0.38      0.47     12828

    accuracy                           0.49     36783
   macro avg       0.45      0.51      0.47     36783
weighted avg       0.51      0.49      0.49     36783

Testing Set Accuracy: 0.4847488038277512
Testing Set Classification Report:
               precision    recall  f1-score   support

           0       0.17      0.43      0.25      1131
           1       0.52      0.66      0.58      2569
           2       0.45      0.48      0.46      2971
           3       0.39      0.44      0.41      7728
           4       0.56  

# Multilayer Perceptrons

In [ ]:
# Define the MLPClassifier model
mlp_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, activation='relu', solver='adam', verbose=True, tol=0.001, alpha=0.0001)

# Fit the model
mlp_model.fit(X_train_prep.iloc[:, :-1], X_train_prep.iloc[:, -1])

Iteration 1, loss = 1.02272142
Iteration 2, loss = 0.87291465
Iteration 3, loss = 0.84069754
Iteration 4, loss = 0.82576398
Iteration 5, loss = 0.80413451
Iteration 6, loss = 0.79037603
Iteration 7, loss = 0.78145145
Iteration 8, loss = 0.76492862
Iteration 9, loss = 0.75439573
Iteration 10, loss = 0.74195922
Iteration 11, loss = 0.73225770
Iteration 12, loss = 0.71465919
Iteration 13, loss = 0.70543931
Iteration 14, loss = 0.69677317
Iteration 15, loss = 0.68062696
Iteration 16, loss = 0.66780370
Iteration 17, loss = 0.65929098
Iteration 18, loss = 0.64130269
Iteration 19, loss = 0.63049533
Iteration 20, loss = 0.61488443
Iteration 21, loss = 0.60732695
Iteration 22, loss = 0.59388270
Iteration 23, loss = 0.58023514
Iteration 24, loss = 0.56895184
Iteration 25, loss = 0.55913778
Iteration 26, loss = 0.54586098
Iteration 27, loss = 0.53710650
Iteration 28, loss = 0.52616913
Iteration 29, loss = 0.51154224
Iteration 30, loss = 0.49922288
Iteration 31, loss = 0.49018056
Iteration 32, los

In [ ]:
# Predict and evaluate
y_val_pred = mlp_model.predict(X_val_prep.iloc[:, :-1])
print("Validation Set Accuracy:", accuracy_score(X_val_prep.iloc[:, -1], y_val_pred))
print("Validation Set Classification Report:\n", classification_report(X_val_prep.iloc[:, -1], y_val_pred))

In [ ]:
# Predict and evaluate
y_test_pred = mlp_model.predict(X_test_prep.iloc[:, :-1])
print("Validation Set Accuracy:", accuracy_score(X_test_prep.iloc[:, -1], y_test_pred))
print("Validation Set Classification Report:\n", classification_report(X_test_prep.iloc[:, -1], y_test_pred))

# Transformer Model

In [ ]:
# Define device upfront
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming X_train_prepared, X_val_prepared, X_test_prepared and their respective y's are already defined

# Encode labels numerically
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "features": torch.tensor(self.features[idx], dtype=torch.float),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Dataset and DataLoader
train_dataset = CustomDataset(X_train_prepared, y_train_encoded)
val_dataset = CustomDataset(X_val_prepared, y_val_encoded)
test_dataset = CustomDataset(X_test_prepared, y_test_encoded)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Model Definition
class Classifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Classifier(input_size=X_train_prepared.shape[1], num_classes=len(set(y_train_encoded)))
model.to(device)

# Training
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        features, labels = batch['features'].to(device), batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    

# Evaluation
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch in test_loader:
        features, labels = batch['features'].to(device), batch['labels'].to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print("Test Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))
